In [ ]:
!git clone https://github.com/ash-105/CREDIFY.git

Cloning into 'CREDIFY'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 111 (delta 9), reused 0 (delta 0), pack-reused 84 (from 1)
Receiving objects: 100% (111/111), 15.62 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00


In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
print(nltk.data.path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [ ]:
nltk.data.path.append('/root/nltk_data')  # Explicitly set the NLTK data path
nltk.download('punkt', download_dir='/root/nltk_data')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Veracity Prediction

In [4]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset, Subset
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
import time

In [11]:
class FactVerificationDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        encoded_input = self.tokenizer.encode_plus(
            item['Claim'],
            item['Most Relevant Sentence'],
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        label = item['label']
        return {
            'input_ids': encoded_input['input_ids'].squeeze(),
            'attention_mask': encoded_input['attention_mask'].squeeze(),
            'label': torch.tensor(label)
        }

In [12]:
# Load the dataset
excel_file = '/content/drive/MyDrive/CREDIFY-main/CREDIFY-main/RECTIFY dataset/sentence.xlsx'
df = pd.read_excel(excel_file)
fields = ['Claim', 'Most Relevant Sentence', 'label']
data_list = df[fields].to_dict(orient='records')

In [13]:
# Set up k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
all_accuracies = []
total_start_time = time.time()

In [15]:
# Cross-validation loop
for fold, (train_idx, test_idx) in enumerate(kf.split(data_list)):
    print(f"\nFold {fold + 1}/5")

    fold_start_time = time.time()

    # Split data into training and test sets
    train_data = [data_list[i] for i in train_idx]
    test_data = [data_list[i] for i in test_idx]

    train_size = int(0.7 * len(train_data))  # 70% for training
    val_size = len(train_data) - train_size  # 15% for validation
    train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_size, val_size])

    train =FactVerificationDataset(train_dataset)
    val =FactVerificationDataset(val_dataset)

    train_dataloader = DataLoader(train, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(FactVerificationDataset(test_data), batch_size=16, shuffle=False)

    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3).to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)

    # Train the model
    num_epochs = 5
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        model.train()
        total_loss = 0

        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        epoch_time = time.time() - epoch_start_time  # Time taken for epoch
        print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {total_loss / len(train_dataloader):.4f} - Time: {epoch_time:.2f}s")

    # Evaluate the model on the validation set
    model.eval()
    correct_predictions = []
    total_predictions = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predicted_labels = torch.argmax(outputs.logits, dim=1)

            correct_predictions.extend(predicted_labels.cpu().numpy().tolist())
            total_predictions.extend(labels.cpu().numpy().tolist())

    val_accuracy = accuracy_score(correct_predictions, total_predictions)
    print(f"Fold {fold + 1} - Validation Accuracy: {val_accuracy:.4f}")

    # Evaluate the model on the test set
    correct_predictions = []
    total_predictions = []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predicted_labels = torch.argmax(outputs.logits, dim=1)

            correct_predictions.extend(predicted_labels.cpu().numpy().tolist())
            total_predictions.extend(labels.cpu().numpy().tolist())

    test_accuracy = accuracy_score(correct_predictions, total_predictions)
    print(f"Fold {fold + 1} - Test Accuracy: {test_accuracy:.4f}")

    all_accuracies.append(test_accuracy)

    fold_time = time.time() - fold_start_time  # Total time per fold
    print(f"Fold {fold + 1} - Total Time: {fold_time:.2f}s")


Fold 1/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5 - Loss: 0.4993 - Time: 247.72s
Epoch 2/5 - Loss: 0.1604 - Time: 251.86s
Epoch 3/5 - Loss: 0.1085 - Time: 251.87s
Epoch 4/5 - Loss: 0.0483 - Time: 251.80s
Epoch 5/5 - Loss: 0.0393 - Time: 251.83s
Fold 1 - Validation Accuracy: 0.9433
Fold 1 - Test Accuracy: 0.9330
Fold 1 - Total Time: 1316.19s

Fold 2/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5 - Loss: 0.5341 - Time: 251.84s
Epoch 2/5 - Loss: 0.1397 - Time: 251.85s
Epoch 3/5 - Loss: 0.0660 - Time: 251.85s
Epoch 4/5 - Loss: 0.0590 - Time: 251.86s
Epoch 5/5 - Loss: 0.0370 - Time: 251.86s
Fold 2 - Validation Accuracy: 0.9458
Fold 2 - Test Accuracy: 0.9290
Fold 2 - Total Time: 1320.22s

Fold 3/5


...

Epoch 1/5 - Loss: 0.5082 - Time: 251.84s
Epoch 2/5 - Loss: 0.1606 - Time: 251.84s
Epoch 3/5 - Loss: 0.1033 - Time: 251.87s
Epoch 4/5 - Loss: 0.0732 - Time: 251.82s
Epoch 5/5 - Loss: 0.0432 - Time: 251.84s
Fold 3 - Validation Accuracy: 0.9167
Fold 3 - Test Accuracy: 0.9310
Fold 3 - Total Time: 1320.26s

Fold 4/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5 - Loss: 0.5270 - Time: 251.83s
Epoch 2/5 - Loss: 0.1677 - Time: 251.69s
Epoch 3/5 - Loss: 0.0909 - Time: 251.60s
Epoch 4/5 - Loss: 0.0613 - Time: 251.83s
Epoch 5/5 - Loss: 0.0412 - Time: 251.77s
Fold 4 - Validation Accuracy: 0.9458
Fold 4 - Test Accuracy: 0.9430
Fold 4 - Total Time: 1319.77s

Fold 5/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5 - Loss: 0.5041 - Time: 251.74s
Epoch 2/5 - Loss: 0.1625 - Time: 251.77s
Epoch 3/5 - Loss: 0.0952 - Time: 251.82s
Epoch 4/5 - Loss: 0.0656 - Time: 251.81s
Epoch 5/5 - Loss: 0.0439 - Time: 251.82s
Fold 5 - Validation Accuracy: 0.9517
Fold 5 - Test Accuracy: 0.9560
Fold 5 - Total Time: 1319.73s


In [16]:
# Total cross-validation time
total_time = time.time() - total_start_time
print(f"\nTotal Cross-validation Time: {total_time:.2f}s")


Total Cross-validation Time: 6642.71s


In [17]:
# Final results
import numpy as np
print("\nCross-validation results:")
print(f"Average Accuracy: {np.mean(all_accuracies):.4f} ± {np.std(all_accuracies):.4f}")


Cross-validation results:
Average Accuracy: 0.9384 ± 0.0100
